In [2]:
from bs4 import BeautifulSoup
from requests_html import AsyncHTMLSession
import requests
import re
import unicodedata
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import json
import pprint
import pickle
import pandas as pd


import asyncio
if asyncio.get_event_loop().is_running(): # Only patch if needed (i.e. running in Notebook, Spyder, etc)
    import nest_asyncio
    nest_asyncio.apply()

# Software Talks Conferences

In [4]:
st_url = 'https://www.softwaretalks.io/'
page = unicodedata.normalize('NFKD', requests.get(st_url).text)
doc = BeautifulSoup(page, 'html.parser')

In [8]:
st_confs = {}
for conference in doc.find_all(class_='conference-item'):
    a_tags = conference.find_all('a')
    
    # Looking at a section
    if len(a_tags) == 0:
        continue
    
    # Title
    title = a_tags[0].text.strip()
    
    # URL
    url = a_tags[1].text
    if url[-1] != '/':
        url = f'{url}/'
    if not 'www' in url:
        url_split = url.split('//')
        url = f'{url_split[0]}//www.{url_split[1]}'
        
    date_online_location = conference.p.text.split('.', 1)
    
    # Date
    date = date_online_location[0].split(' ')
    start = datetime.strptime(date[1], '%d/%m/%Y')
    end = datetime.strptime(date[3], '%d/%m/%Y')
    
    online_location = date_online_location[1]
    
    # Online
    online = True if 'Online' in online_location else None
    
    # Location
    city = None
    state = None
    country = None
    if online:
        location = online_location.split('Online')[0].strip()
    else:
        location = online_location.strip()
    if location != '':
        city_state_country = location.split(',')
        city = city_state_country[0]
        if (len(city_state_country[1].strip()) == 2):
            state = city_state_country[1]
            country = 'United States'
        else:
            country = city_state_country[1]
    
    # Topics
    topics = []
    
    # Normalize across all lists
    if country == 'Czechia':
        country = 'Czech Republic'
        
        
    st_confs[f'{title} -- {str(start).split(" ")[0]}'] = {
        'title': title.strip(),
        'url': url.strip(),
        'start': start,
        'end': end,
        'online': online,
        'topics': topics,
        'country': country.strip() if country else country,
        'city': city.strip() if city else city,
        'state': state.strip() if state else state,
        'source': ['SoftwareTalks'],
    }

# Confs Tech Conferences

In [12]:
confs_tech_url = 'https://confs.tech/#'
asession = AsyncHTMLSession()
response = await asession.get(confs_tech_url)
await response.html.arender()
html = unicodedata.normalize('NFKD', response.html.html)
doc = BeautifulSoup(html, 'html.parser')

In [29]:
ct_confs = {}
for conference in doc.find_all('li', class_='ConferenceItem_ConferenceItem__Hnn7O'):
    data = json.loads(conference.find('script', type='application/ld+json').text)
    
    # Title5m
    title = data['name']
    
    # URL
    url = data['url']
    
    # Date
    start = datetime.strptime(data['startDate'], '%Y-%m-%d')
    end = datetime.strptime(data['endDate'], '%Y-%m-%d')
    
    print(start)
    break

2023-03-18 00:00:00
